# 1 Introduction

This notebook explores the scalability and performance characteristics of working with large analytical datasets using modern data formats and processing engines. Focusing on **CSV** versus **Parquet (Arrow)** formats, it demonstrates how efficient storage layouts—combined with high-performance local query engines such as **DuckDB** and fast DataFrame libraries like **Polars** can significantly improve query speed, memory usage, and overall workflow efficiency.

To structure this exploration, we will:

- Load the MovieLens datasets in their original **CSV** format

- Convert the datasets into the **Parquet (Arrow)** format to benefit from columnar storage.

- Create and query **Parquet**-backed tables in **DuckDB**.

- Execute benchmark tests using both **DuckDB** and **Polars** across multiple scenarios.

- Compare performance metrics as dataset size and query complexity increase.

Through these steps, we aim to highlight how modern columnar formats and vectorized engines scale under more demanding analytical workloads. In particular, we expect **Polars** and **Parquet** to deliver increasingly superior performance—both in execution time and resource efficiency as data volume grows and queries become more complex.

# 2 Library import, Data and files import, duckdb import and conversion of files

In [1]:
#   BLOCO INICIAL — IMPORTS & PATHS & SETUP

import duckdb
import pandas as pd
import polars as pl
import time
from pathlib import Path

# === PATHS FOR THE DATASETS ===

## MovieLens 100k
DATA_100k = Path("..") / "data" / "100k"
ratings_100k_csv = DATA_100k / "ratings.csv"
movies_100k_csv  = DATA_100k / "movies.csv"
tags_100k_csv    = DATA_100k / "tags.csv"
links_100k_csv   = DATA_100k / "links.csv"

## MovieLens 33M
DATA_33m = Path("..") / "data" / "Full33M"
ratings_33m_csv = DATA_33m / "ratings.csv"
movies_33m_csv  = DATA_33m / "movies.csv"
tags_33m_csv    = DATA_33m / "tags.csv"
links_33m_csv   = DATA_33m / "links.csv"

print("Paths defined successfully.")

# === Local DuckDB connection ===
con = duckdb.connect("movielens_local.duckdb")
print("DuckDB connection opened.")

Paths defined successfully.
DuckDB connection opened.


### 1.2 Create Parquet Tables in DuckDB (ratings_parquet, movies_parquet)

In [2]:
# === CONVERSÃO CSV -> PARQUET  ===

ratings_100k_parquet = ratings_100k_csv.with_suffix(".parquet")
ratings_33m_parquet  = ratings_33m_csv.with_suffix(".parquet")
movies_33m_parquet   = movies_33m_csv.with_suffix(".parquet")

print("Parquet paths:")
print(ratings_100k_parquet)
print(ratings_33m_parquet)
print(movies_33m_parquet)

# Converter 
duckdb.sql(f"""
COPY (SELECT * FROM read_csv_auto('{ratings_100k_csv}'))
TO '{ratings_100k_parquet}'
(FORMAT PARQUET);
""")

duckdb.sql(f"""
COPY (SELECT * FROM read_csv_auto('{ratings_33m_csv}'))
TO '{ratings_33m_parquet}'
(FORMAT PARQUET);
""")

duckdb.sql(f"""
COPY (SELECT * FROM read_csv_auto('{movies_33m_csv}'))
TO '{movies_33m_parquet}'
(FORMAT PARQUET);
""")

print("CSV -> Parquet conversion finished.")


Parquet paths:
..\data\100k\ratings.parquet
..\data\Full33M\ratings.parquet
..\data\Full33M\movies.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CSV -> Parquet conversion finished.


In [3]:
# List of all tables in the database
con.sql("""
SELECT table_name, table_type
FROM information_schema.tables
""").df()

,table_name,table_type


In [4]:
print(ratings_100k_csv)
print(ratings_33m_csv)

..\data\100k\ratings.csv
..\data\Full33M\ratings.csv


# 3 Performance test and measuremnts
## 3.1 Fuction to measure time on queries

In [5]:
# Função medir_tempo

def medir_tempo(query):
    t0 = time.time()
    duckdb.sql(query).df()
    return round(time.time() - t0, 3)

## 3.2 Test with simple Query in file with movieId: CSV 100k vs PARQUET 100k and CSV with 33M vs PARQUET with 33M

### Benchmark 1 — CSV vs Parquet Performance (AVG ratings + GROUP BY moviesId)

We measure the execution time of a simple aggregation query (AVG(rating) GROUP BY movieId) executed using DuckDB, running on both CSV and Parquet files for the MovieLens 100k and 33M datasets. This benchmark isolates the impact of the storage format on query performance within DuckDB.

In [6]:
# Medições CSV vs Parquet

tempos = {
    "CSV_100k": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_100k_csv}'
        GROUP BY movieId
    """),

    "PARQUET_100k": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_100k_parquet}'
        GROUP BY movieId
    """),

    "CSV_33M": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_33m_csv}'
        GROUP BY movieId
    """),

    "PARQUET_33M": medir_tempo(f"""
        SELECT movieId, AVG(rating)
        FROM '{ratings_33m_parquet}'
        GROUP BY movieId
    """),
}

tempos


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

{'CSV_100k': 0.764,
 'PARQUET_100k': 0.101,
 'CSV_33M': 7.435,
 'PARQUET_33M': 0.89}

In [7]:
# Tabela comparativa

df_tempo = pd.DataFrame([
    ["100k", tempos["CSV_100k"], tempos["PARQUET_100k"]],
    ["33M", tempos["CSV_33M"], tempos["PARQUET_33M"]],
], columns=["Dataset", "CSV time (s)", "Parquet time (s)"])

df_tempo


,Dataset,CSV time (s),Parquet time (s)
0,100k,0.764,0.101
1,33M,7.435,0.890


__Conclusion:__

This benchmark measures the execution time of a simple aggregation query (`AVG(rating)` grouped by `movieId`) across four scenarios:

- CSV 100K
- Parquet 100K
- CSV 33M
- Parquet 33M

The results show the strong impact of storage format on query speed, with Parquet providing substantial improvements—especially at larger scales.


# 4.0 DuckDB vs Polars

## 4.1 Benchmark 2 — Simple COUNT() Query (DuckDB vs Polars)

Here we run the same aggregation query (`AVG(rating)` grouped by `movieId`) on Parquet files using DuckDB and Polars, for 100k and 33M rows, in order to compare the execution engines under a simple workload.

In [8]:
# paths para Parquet

p100k = ratings_100k_csv.with_suffix(".parquet")
p33m = ratings_33m_csv.with_suffix(".parquet")

In [9]:
# Funções de benchmark

def run_duckdb(path):
    t0 = time.time()
    duckdb.sql(f"""
        SELECT movieId, AVG(rating)
        FROM '{path}'
        GROUP BY movieId
    """).df()
    return round(time.time() - t0, 3)

def run_polars(path):
    t0 = time.time()
    (
        pl.scan_parquet(str(path))     
          .group_by("movieId")         
          .agg(pl.col("rating").mean())
          .collect()
    )
    return round(time.time() - t0, 3)


In [10]:
# Comparação

df_duck_polars = pd.DataFrame([
    ["100k", run_duckdb(p100k), run_polars(p100k)],
    ["33m", run_duckdb(p33m), run_polars(p33m)],
], columns=["Dataset", "DuckDB (s)", "Polars (s)"])

df_duck_polars


,Dataset,DuckDB (s),Polars (s)
0,100k,0.018,0.040
1,33m,0.698,1.828


## 4.2 Benchmark 3 — Aggregation + JOIN (AVG, COUNT, STDDEV) — DuckDB vs Polars

This benchmark uses a more realistic analytical workload on the 33M dataset:
we compute average rating, number of ratings and standard deviation per movie, join with the movies table to retrieve titles, filter movies with at least 500 ratings, and sort by total_ratings (TOP 100). The goal is to compare DuckDB and Polars under a heavier analytical query.

**DuckDB**

In [11]:
# paths para Parquet (AGORA como string)
ratings_33m_parquet = ratings_33m_csv.with_suffix(".parquet")
movies_33m_parquet  = movies_33m_csv.with_suffix(".parquet")

In [12]:
MIN_RATINGS = 500

def run_duckdb_movie_stats(con):
    sql = f"""
    WITH movie_stats AS (
        SELECT
            m.movieId,
            m.title,
            AVG(r.rating) AS avg_rating,
            COUNT(*) AS total_ratings,
            STDDEV_POP(r.rating) AS std_rating
        FROM read_parquet('{ratings_33m_parquet.as_posix()}') r
        JOIN read_parquet('{movies_33m_parquet.as_posix()}') m USING (movieId)
        GROUP BY m.movieId, m.title
        HAVING COUNT(*) >= {MIN_RATINGS}
    )
    SELECT
        movieId,
        title,
        ROUND(avg_rating, 3) AS avg_rating,
        total_ratings,
        ROUND(std_rating, 3) AS std_rating
    FROM movie_stats
    ORDER BY total_ratings DESC
    LIMIT 100
    """
    return con.sql(sql).df()


In [13]:
start = time.perf_counter()
df_duckdb_stats = run_duckdb_movie_stats(con)
t_duckdb_stats = time.perf_counter() - start



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
import time

start = time.perf_counter()
df_duckdb_stats = run_duckdb_movie_stats(con)
t_duckdb_stats = time.perf_counter() - start
#df_duckdb_stats.head(), t_duckdb_stats


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

**Polars**

In [15]:
def run_polars_movie_stats_33m():
    start = time.perf_counter()

    # usar os ficheiros PARQUET, não os CSV
    ratings = (
        pl.scan_parquet(str(ratings_33m_parquet))
        .select(["movieId", "rating"])
    )

    movies  = (
        pl.scan_parquet(str(movies_33m_parquet))
        .select(["movieId", "title"])
    )

    MIN_RATINGS = 500

    result = (
        ratings
        .group_by("movieId")
        .agg([
            pl.col("rating").mean().alias("avg_rating"),
            pl.count().alias("total_ratings"),
            pl.col("rating").std().alias("std_rating"),
        ])
        .filter(pl.col("total_ratings") >= MIN_RATINGS)
        .join(movies, on="movieId")
        .select([
            "movieId",
            "title",
            pl.col("avg_rating").round(3),
            "total_ratings",
            pl.col("std_rating").round(3),
        ])
        .sort("total_ratings", descending=True)
        .limit(100)
        .collect()
    )

    elapsed = time.perf_counter() - start
    return result, elapsed


In [16]:
df_polars_stats_33m, t_polars_stats_33m = run_polars_movie_stats_33m()
#df_polars_stats_33m.head(), t_polars_stats_33m


C:\Users\sarac\AppData\Local\Temp\ipykernel_5312\1360028000.py:22: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("total_ratings"),


In [17]:
df_complex = pd.DataFrame([
    ["33M (complex)", t_duckdb_stats, t_polars_stats_33m],
], columns=["Dataset", "DuckDB (s)", "Polars (s)"])

df_complex


,Dataset,DuckDB (s),Polars (s)
0,33M (complex),3.380861,2.221839


# 5.0 Summary

In [18]:
df_summary = pd.DataFrame([
    ["1","100k","CSV vs Parquet", "DuckDB_CSV", tempos["CSV_100k"]],
    ["1","100k","CSV vs Parquet", "DuckDB_PARQUET", tempos["PARQUET_100k"]],
    ["1","33M","CSV vs Parquet", "DuckDB_CSV", tempos["CSV_33M"]],
    ["1","33M","CSV vs Parquet", "DuckDB_PARQUET", tempos["PARQUET_33M"]],
    ["2","100k","Simple Test", "DuckDB", df_duck_polars.loc[0, "DuckDB (s)"]],
    ["2","100k","Simple Test", "Polars", df_duck_polars.loc[0, "Polars (s)"]],
    ["2","33M","Simple Test", "DuckDB", df_duck_polars.loc[1, "DuckDB (s)"]],
    ["3","33M","Simple Test", "Polars", df_duck_polars.loc[1, "Polars (s)"]],
    ["3","33M","Complex Test", "DuckDB", df_complex.loc[0, "DuckDB (s)"]],
    ["3","33M","Complex Test", "Polars", df_complex.loc[0, "Polars (s)"]],
], columns=["Benchmark","Dataset","Scenario", "Engine", "Time (s)"])

df_summary


,Benchmark,Dataset,Scenario,Engine,Time (s)
0,1,100k,CSV vs Parquet,DuckDB_CSV,0.764000
1,1,100k,CSV vs Parquet,DuckDB_PARQUET,0.101000
2,1,33M,CSV vs Parquet,DuckDB_CSV,7.435000
3,1,33M,CSV vs Parquet,DuckDB_PARQUET,0.890000
4,2,100k,Simple Test,DuckDB,0.018000
5,2,100k,Simple Test,Polars,0.040000
6,2,33M,Simple Test,DuckDB,0.698000
7,3,33M,Simple Test,Polars,1.828000
8,3,33M,Complex Test,DuckDB,3.380861
9,3,33M,Complex Test,Polars,2.221839


The results show that Parquet is substantially faster than CSV, delivering up to a considerable improvement on the 33M dataset.

In the engine comparison (Benchmark 2), DuckDB is consistently faster on simple aggregations, both on the 100k dataset (0.018s vs 0.040s) and the 33M dataset (0.698s vs 1.828s), while Polars remains competitive.

In the complex analytical query (Benchmark 3), Polars outperforms DuckDB on the 33M dataset (2.22s vs 3.38s), demonstrating better scalability under heavier workloads.

Both engines achieve very efficient performance when operating on Parquet, highlighting the benefits of columnar storage for analytical processing.

Overall, Parquet + Polars provides the strongest performance for large-scale analytical queries as dataset size and query complexity increase.

__Close the connection (when done)__

In [19]:
#con.close()
#print("Connection closed.")